In [316]:
import requests
from bs4 import BeautifulSoup
import json
import re
from urllib.parse import urljoin

In [ ]:
def get_all_links(url, visited):
    """Obtiene todos los enlaces de una página web y los filtra para evitar bucles."""
    links = set()
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            for a_tag in soup.find_all('a', href=True):
                full_url = urljoin(url, a_tag['href'])
                if full_url.startswith(url) and full_url not in visited:
                    links.add(full_url)
    except Exception as e:
        print(f"Error obteniendo enlaces de {url}: {e}")
    return links if links else set()

In [318]:
def clean_text(text):
    """Elimina caracteres especiales y basura del texto extraído."""
    # Mantener solo caracteres ASCII imprimibles y eliminar otros caracteres no deseados
    cleaned_text = re.sub(r'[^\x20-\x7E\n]', '', text)  # Mantiene solo caracteres ASCII imprimibles
    # Filtra cadenas muy largas con caracteres no alfanuméricos
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s.,;!?@#$%^&*()_+-=<>/\\]{20,}', '', cleaned_text)
    return cleaned_text


In [319]:
def extract_text(url):
    """Extrae todo el texto de una página web y lo limpia de caracteres extraños."""
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return None

        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text(separator='\n', strip=True)
        return clean_text(text)
    except Exception as e:
        print(f"Error extrayendo información de {url}: {e}")
        return None

In [320]:
import re

def categorize_text(text):
    """Clasifica el texto en categorías según palabras clave específicas."""
    categories = {
        "Contacto": [],
        "Páginas Web": [],
        "Ubicación": [],
        "Educación": []  # ✅ Guardará líneas completas solo si contienen una palabra clave exacta
    }
    
    # Expresión regular para números de teléfono o fax (9 dígitos)
    phone_fax_regex = r'\b\d{9}\b'
    
    # Expresión regular para correos electrónicos con @gmail, @gva.es o @edu.gva
    email_regex = r'\b[A-Za-z0-9._%+-]+@(gmail\.com|gva\.es|edu\.gva)\b'
    
    # Expresión regular para detectar términos relacionados con ubicaciones
    location_keywords = r'\b(Adreça|Calle|Carrer|Avenida|Avda|Codi Postal|Ciutat|Ciudad|Adrea)\b'
    
    # Lista de palabras clave exactas para información educativa
    education_keywords = {"ESO", "Educación Secundaria", "Bachillerato", "FP", 
                          "Formación Profesional", "Batxillerat", "Superior", "Artificial"}
    
    for line in text.split('\n'):
        # Buscar números de teléfono o fax
        if re.search(phone_fax_regex, line):
            categories["Contacto"].append(line)
        # Buscar correos electrónicos con las extensiones permitidas
        elif re.search(email_regex, line, re.IGNORECASE):
            categories["Contacto"].append(line)
        # Buscar páginas web (enlaces)
        elif re.match(r'^https?://', line):
            categories["Páginas Web"].append(line)
        # Buscar información relacionada con la ubicación
        elif re.search(location_keywords, line, re.IGNORECASE):
            categories["Ubicación"].append(line)
        # Buscar información educativa y almacenar la línea completa solo si tiene una palabra clave exacta
        else:
            words = set(re.findall(r'\b\w+\b', line))  # Extraer palabras sin signos de puntuación
            if words & education_keywords:  # Si hay intersección con palabras clave, guardamos la línea
                categories["Educación"].append(line)
    
    return categories


In [321]:
def scrape_website(base_url):
    """Extrae toda la información de una página web y sus enlaces y la organiza por categorías."""
    visited = set()
    to_visit = {base_url}
    categorized_data = {
        "Contacto": set(),
        "Páginas Web": set(),
        "Ubicación": set(),
        "Educación": set()  
    }
    
    while to_visit:
        url = to_visit.pop()
        visited.add(url)
        
        text = extract_text(url)
        if text:
            page_data = categorize_text(text)
            for category, content in page_data.items():
                if category in categorized_data:  
                    categorized_data[category].update(content)
        
        to_visit.update(get_all_links(url, visited) or set())
    
    intents = [
        {"tag": category, "patterns": [f"¿Qué información hay sobre {category.lower()}?"], "responses": list(content)}
        for category, content in categorized_data.items() if content
    ]
    
    return {"intents": intents}


In [322]:
def save_to_json(data, filename="output.json"):
    """Guarda la información en un archivo JSON."""
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)


In [323]:
url = "https://portal.edu.gva.es/iesserraperenxisa/"
scraped_data = scrape_website(url)
save_to_json(scraped_data)
print("Datos guardados en output.json")

Datos guardados en output.json
